In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [20]:
df = pd.read_csv('/content/train.csv')

In [21]:
df.drop('Cabin', inplace=True, axis=1)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C


In [22]:
df.drop(['Name', 'PassengerId', 'Ticket'], axis=1, inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [23]:
df['family_size'] = df.SibSp + df.Parch+1

In [24]:
df.drop(['SibSp', 'Parch'], axis=1, inplace=True)
df

,Survived,Pclass,Sex,Age,Fare,Embarked,family_size
0,0,3,male,22.0,7.2500,S,2
1,1,1,female,38.0,71.2833,C,2
2,1,3,female,26.0,7.9250,S,1
3,1,1,female,35.0,53.1000,S,2
4,0,3,male,35.0,8.0500,S,1
...,...,...,...,...,...,...,...
886,0,2,male,27.0,13.0000,S,1
887,1,1,female,19.0,30.0000,S,1
888,0,3,female,NaN,23.4500,S,4
889,1,1,male,26.0,30.0000,C,1


In [25]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df['Sex'] = label_encoder.fit_transform(df['Sex'])
df['Embarked'] = label_encoder.fit_transform(df['Embarked'].astype(str))
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [26]:
X=df.drop('Survived', axis=1)
y=df.Survived

In [27]:
X

,Pclass,Sex,Age,Fare,Embarked,family_size
0,3,1,22.0,7.2500,2,2
1,1,0,38.0,71.2833,0,2
2,3,0,26.0,7.9250,2,1
3,1,0,35.0,53.1000,2,2
4,3,1,35.0,8.0500,2,1
...,...,...,...,...,...,...
886,2,1,27.0,13.0000,2,1
887,1,0,19.0,30.0000,2,1
888,3,0,28.0,23.4500,2,4
889,1,1,26.0,30.0000,0,1


In [28]:
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [30]:
def Euclidean(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def knn(train_data, train_labels, test_data, k):
    predictions = []
    for i in range(len(test_data)):
        distances = []
        for j in range(len(train_data)):
            distance = Euclidean(test_data.iloc[i].values, train_data.iloc[j].values)
            distances.append((distance, train_labels.iloc[j]))
        distances.sort(key=lambda x: x[0])
        neighbors = distances[:k]
        survived = sum([1 for i in neighbors if i[1] == 1])
        not_survived = sum([1 for i in neighbors if i[1] == 0])
        if survived > not_survived:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

In [31]:
k = 5
predictions = knn(X_train, y_train, X_test, k)

accuracy = sum(predictions == y_test) / len(y_test) * 100
print(f"Accuracy: {accuracy}%")

Accuracy: 73.18435754189943%


In [32]:
ch1 = predictions.count(1)
ch2 = 0
for i in range(len(predictions)):
  if y_test.iloc[i] == 1:
    ch2 = ch2 + 1
TP = 0
for i in range(len(predictions)):
  if predictions[i]==y_test.iloc[i]==1:
    TP=TP+1
FP = abs(ch2-TP)
FN = abs(ch1-TP)
TN = len(predictions)-(TP+FP+FN)
print("TP =",TP,"\nFP =",FP,"\nFN =",FN,"\nTN =",TN)
print("Accuracy = ",(TP+TN)/(TP+TN+FP+FN))

TP = 41 
FP = 33 
FN = 15 
TN = 90
Accuracy =  0.7318435754189944


In [33]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,predictions)
print(cm)

[[90 15]
 [33 41]]
